In [ ]:
import MRGG
from MRGG.Graph import Graph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import os
import time
import math

from compute_errors import compute_errors
from hypothesis_testing import hypothesis_testing
from link_probas import link_probas
from risks import risks

## 1) Function used to generate results

- Hypothesis Testing: we test if the latent positions have been sampled uniformly or not.
- Computation of the Beysian, MRGG and IID risks.
- Computation of delta_2 errors between the spectra of the estimated and the true envelope or latitude.
- Computation of the L2 errors between the estimated and the true envelope or latitude.
- Probabilities of connections between the ten first nodes in the graph and the last node in the graph.

In [ ]:
def parallel_computations(dimension, seed, list_n, envelope, latitude, path, params):
    import os
    import time
    os.chdir('MRGG')
    from compute_errors import compute_errors
    from link_probas import link_probas
    from hypothesis_testing import hypothesis_testing
    from risks import risks
    try:
        os.mkdir(path+str(seed))
    except:
        pass
    hypothesis_testing(dimension, seed, list_n=list_n, envelope=envelope, latitude=latitude, path=path+str(seed)+'/', params=params)
    print('Hypothesis Testing done: ', time.time())
    risks(dimension, seed, list_n=list_n, envelope=envelope, latitude=latitude, path=path+str(seed)+'/', params=params)
    print('Risk done: ', time.time())
    compute_errors(dimension, seed, list_n=list_n, envelope=envelope, latitude=latitude, path=path+str(seed)+'/', params=params)
    print('Errors done: ', time.time())
    link_probas(dimension, seed, list_n=list_n, nbnodes=10, envelope=envelope, latitude=latitude, path=path+str(seed)+'/', params=params)
    print('Proba done: ', time.time())

## 2) Definition of the settings for which we want to compute results

In [ ]:
dims = 12*[4]
seeds = [i for i in range(12)]
lists_n = [[200,500,800,1100,1500] for i in range(12)]
envs = 8*['rayleigh']+4*['heaviside']
lats = 3*(3*['beta']+['mixture'])
paths = 12*['final_data/']

envparams = [{'xi':0.5,'eta':1,'r':1},{'xi':0.25,'eta':3,'r':1},{}]
latparams = [{'alpha':1,'beta':3},{'alpha':2,'beta':5},{'alpha':2,'beta':2},{}]
params = [ dict(envdic, **latdic) for envdic in envparams for latdic in latparams]
print(params)

## 3) Parallel computations of the results

In [ ]:
import ipyparallel

# attach to a running cluster
cluster = ipyparallel.Client()
print('profile:', cluster.profile)
print("IDs:", cluster.ids) # Print process id numbers

%px import socket
%px print("hosts:", socket.gethostname())

cluster[:].map_sync(parallel_computations, dims, seeds, lists_n, envs, lats, paths, params)


## 4) Generation of figures for saved files

In [ ]:
for i in range(12):
    plt.figure(i)
    a = np.load('final_data/'+str(i)+'/link_probas_iid_n1500_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')
    plt.scatter([i for i in range(10)], a, marker='*', label='iid case (RGG estimate)',s=100*np.ones(10))
    a = np.load('final_data/'+str(i)+'/true_link_probas_n1500_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')
    plt.scatter([i for i in range(10)], a, marker='+', label='Bayes optimal probabilities',s=100*np.ones(10))
    a = np.load('final_data/'+str(i)+'/link_probas_pred_n1500_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')
    plt.scatter([i for i in range(10)], a,marker='+', c='green', label='MRGG estimate',s=100*np.ones(10))
    plt.legend(fontsize=13)
    plt.xlabel('$10$ first nodes: $X_1, \dots, X_{10}$',fontsize=14)
    plt.ylabel('Link probability', fontsize=14)

### 4.1) Figure for the Risks

In [ ]:
for i in range(12):
    plt.figure(i)
    risks = np.load('final_data/'+str(i)+'/RISKS_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')

    Liid = []
    L = []
    Lesti = []
    list_n = [200,500,800,1100,1500]
    NBRISK = 30
    limn = len(list_n)
    for i in range(limn):
        Liid.append(np.mean(risks[i,:,1])/NBRISK)
        L.append(np.mean(risks[i,:,0])/NBRISK)
        Lesti.append(np.mean(risks[i,:,2])/NBRISK)

    plt.scatter(list_n, Liid[:limn], marker='*',label='Random classifier',s=100*np.ones(limn))
    plt.scatter(list_n, np.array(Lesti[:limn]),marker='+', c='green', label='MRGG classifier',s=100*np.ones(limn))
    plt.scatter(list_n, np.array(L[:limn]),marker='x', c='orange', label='Bayes classifier',s=100*np.ones(limn))
    plt.xlabel('Size of the graph: $n$',fontsize=14)
    plt.ylabel('Risk',fontsize=14)
    plt.legend(fontsize=13)
    plt.tight_layout()

### 4.2) Figures for the $L^2$ errors

In [ ]:
for i in range(12):
    plt.figure(i)
    list_n = lists_n[i]
    L2env = np.load('final_data/'+str(i)+'/L2errors_env_dim_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')

    L2lat = np.load('final_data/'+str(i)+'/L2errors_lat_dim_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')

    plt.xlabel('size of the graph: $n$',fontsize=14)
    plt.ylabel('$L^2$ errors', fontsize=14)
    plt.plot(list_n, np.mean(L2lat,axis=0), label='Latitude')
    plt.plot(list_n, np.mean(L2env, axis=0), label='Envelope')
    plt.legend(fontsize=13)

### 4.3) Figures for the $\delta_2$ errors

In [ ]:
for i in range(12):
    plt.figure(i)
    L2lat = np.load('final_data/'+str(i)+'/delta2errors_lat_dim_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')
    L2env = np.load('final_data/'+str(i)+'/delta2errors_env_dim_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')


    plt.xlabel('size of the graph: $n$',fontsize=14)
    plt.ylabel('$\delta_2$ errors', fontsize=14)
    plt.plot(list_n, np.mean(L2lat,axis=0), label='Latitude')
    plt.plot(list_n, np.mean(L2env, axis=0), label='Envelope')
    plt.legend(fontsize=13)

### 4.4) Figures for the Hupothesis Testing

In [ ]:

import matplotlib.pyplot as plt
for i in range(12):
    pvalsiid = np.load('final_data/'+str(i)+'/pvalue_iid_dim_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')
    
    pvals = np.load('final_data/'+str(i)+'/pvalue_lat_dim_4_'+envs[i]+'_'+lats[i]+'_'+str(i)+'.npy')

    rejet_iid = np.zeros(len(list_n))
    rejet_lat = np.zeros(len(list_n))
    for k,n in enumerate(list_n):
        sorted_piid = np.sort(pvalsiid[:,k])
        limitechi2 = sorted_piid[int(0.05 * len(sorted_piid))]
        rejet_iid[k] = (int(0.05 * len(sorted_piid))/len(sorted_piid))
        rejet_lat[k] = np.mean( pvals[:,k] < limitechi2 )


    fig = plt.figure(i)
    plt.plot(list_n, rejet_iid, label='Null (Type I error)', linestyle='--')
    plt.plot(list_n, rejet_lat, label='Alternative (Power)')
    plt.xlabel('$n$',fontsize=16)
    plt.ylabel('% of rejection',fontsize=16)
    plt.legend(fontsize=13)